In [ ]:
import warnings
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from dotenv import load_dotenv
from datetime import datetime, timezone

warnings.filterwarnings("ignore")
load_dotenv()

from core.data_sources import CLOBDataSource
from core.notifiers import NotificationManager, NotificationMessage

clob = CLOBDataSource()
notification_manager = NotificationManager()

print("✅ Modules loaded successfully")
print(f"🔔 Enabled notifiers: {', '.join(notification_manager.get_enabled_notifiers())}")

In [ ]:
# Get current order book snapshot for HBOT-USDT on Bitrue
order_book = await clob.get_order_book_snapshot(
    connector_name="bitrue",
    trading_pair="HBOT-USDT",
    depth=50  # Get top 20 levels
)

print(f"Order book snapshot for {order_book['trading_pair']}")
print(f"Timestamp: {pd.to_datetime(order_book['timestamp'], unit='s')}")
print("\n" + "="*50)

# Convert to DataFrames for better visualization
bids_df = pd.DataFrame(order_book['bids'], columns=['price', 'amount'])
asks_df = pd.DataFrame(order_book['asks'], columns=['price', 'amount'])

bids_df['total'] = bids_df['amount'].cumsum()
asks_df['total'] = asks_df['amount'].cumsum()
bids_df['value'] = bids_df['price'] * bids_df['amount']
asks_df['value'] = asks_df['price'] * asks_df['amount']

print(f"\nTop 10 Bids:")
print(bids_df.head(10))

print(f"\nTop 10 Asks:")
print(asks_df.head(10))

# Calculate spread
best_bid = bids_df.iloc[0]['price']
best_ask = asks_df.iloc[0]['price']
spread = best_ask - best_bid
spread_pct = (spread / best_bid) * 100

print(f"\n{'='*50}")
print(f"Best Bid: ${best_bid:.6f}")
print(f"Best Ask: ${best_ask:.6f}")
print(f"Spread: ${spread:.6f} ({spread_pct:.3f}%)")
print(f"Mid Price: ${(best_bid + best_ask) / 2:.6f}")
print(f"\nTotal Bid Liquidity (20 levels): {bids_df['amount'].sum():.2f} HBOT (${bids_df['value'].sum():.2f})")
print(f"Total Ask Liquidity (20 levels): {asks_df['amount'].sum():.2f} HBOT (${asks_df['value'].sum():.2f})")

In [ ]:
# Visualize order book depth with +/- 80% range from mid price
mid_price = (best_bid + best_ask) / 2
price_range_lower = mid_price * 0.2
price_range_upper = mid_price * 1.8

# Filter bids and asks within the price range
bids_filtered = bids_df[bids_df['price'] >= price_range_lower].copy()
asks_filtered = asks_df[asks_df['price'] <= price_range_upper].copy()

# Create single plot for order book
fig = go.Figure()

# Add cumulative liquidity
fig.add_trace(
    go.Scatter(x=bids_filtered['price'], y=bids_filtered['total'], name='Cumulative Bids',
               line=dict(color='green', width=2), fill='tozeroy')
)
fig.add_trace(
    go.Scatter(x=asks_filtered['price'], y=asks_filtered['total'], name='Cumulative Asks',
               line=dict(color='red', width=2), fill='tozeroy')
)

fig.update_xaxes(title_text="Price (USDT)")
fig.update_yaxes(title_text="Cumulative Amount (HBOT)")

fig.update_layout(
    height=600,
    title_text=f"HBOT-USDT Order Book (±80% from mid price)<br><sub>Mid Price: ${mid_price:.6f} | Spread: ${spread:.6f} ({spread_pct:.3f}%)</sub>",
    showlegend=True
)

fig.show()

# Save chart as PNG for Telegram
orderbook_png_path = '/tmp/hbot_orderbook.png'
print("💾 Saving order book chart...")
try:
    fig.write_image(orderbook_png_path, width=1000, height=600, scale=2)
    print(f"✅ Order book chart saved: {orderbook_png_path}")
except Exception as e:
    print(f"❌ Error saving chart: {e}")
    print("💡 Make sure kaleido is installed: pip install kaleido")

In [ ]:
# Uniswap V3 Pool Data Retrieval using web3.py
from web3 import Web3

# HBOT Token: 0xE5097D9baeAFB89f9bcB78C9290d545dB5f9e9CB
HBOT_TOKEN = "0xE5097D9baeAFB89f9bcB78C9290d545dB5f9e9CB"
WETH_TOKEN = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
USDC_TOKEN = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"

# Connect to Ethereum mainnet using RPC from .env
ETH_RPC_URL = os.getenv("ETHEREUM_RPC_URL", "https://eth.llamarpc.com")
w3 = Web3(Web3.HTTPProvider(ETH_RPC_URL))

print(f"Connected to Ethereum: {w3.is_connected()}")
print(f"Latest block: {w3.eth.block_number}")
print(f"Using RPC: {ETH_RPC_URL}")

In [ ]:
# Discover Uniswap V3 Pools for HBOT/WETH and HBOT/USDC
UNISWAP_V3_FACTORY = "0x1F98431c8aD98523631AE4a59f267346ea31F984"

# Uniswap V3 Factory ABI
factory_abi = [
    {
        "inputs": [
            {"internalType": "address", "name": "tokenA", "type": "address"},
            {"internalType": "address", "name": "tokenB", "type": "address"},
            {"internalType": "uint24", "name": "fee", "type": "uint24"}
        ],
        "name": "getPool",
        "outputs": [{"internalType": "address", "name": "pool", "type": "address"}],
        "stateMutability": "view",
        "type": "function"
    }
]

factory = w3.eth.contract(address=Web3.to_checksum_address(UNISWAP_V3_FACTORY), abi=factory_abi)

# Check common fee tiers (0.05%, 0.3%, 1%)
fee_tiers = [500, 3000, 10000]
all_pools = []

# Check HBOT/WETH pools
print("Searching for HBOT/WETH Uniswap V3 pools...")
for fee in fee_tiers:
    try:
        pool_address = factory.functions.getPool(
            Web3.to_checksum_address(HBOT_TOKEN),
            Web3.to_checksum_address(WETH_TOKEN),
            fee
        ).call()

        if pool_address != "0x0000000000000000000000000000000000000000":
            all_pools.append({
                "pair": "HBOT/WETH",
                "fee": fee,
                "address": pool_address
            })
            print(f"  ✓ Found at {fee/10000}% fee tier: {pool_address}")
    except Exception as e:
        print(f"  ✗ Error at {fee/10000}% fee tier: {e}")

# Check HBOT/USDC pools (only 0.05% and 0.3%, skip 1%)
print("\nSearching for HBOT/USDC Uniswap V3 pools...")
for fee in [500, 3000]:  # Skip 10000 (1%)
    try:
        pool_address = factory.functions.getPool(
            Web3.to_checksum_address(HBOT_TOKEN),
            Web3.to_checksum_address(USDC_TOKEN),
            fee
        ).call()

        if pool_address != "0x0000000000000000000000000000000000000000":
            all_pools.append({
                "pair": "HBOT/USDC",
                "fee": fee,
                "address": pool_address
            })
            print(f"  ✓ Found at {fee/10000}% fee tier: {pool_address}")
    except Exception as e:
        print(f"  ✗ Error at {fee/10000}% fee tier: {e}")

print(f"\n{'='*60}")
print(f"Total pools found: {len(all_pools)}")
for pool in all_pools:
    print(f"  {pool['pair']} @ {pool['fee']/10000}% - {pool['address']}")
print(f"{'='*60}")

# Fetch ETH price dynamically from CoinGecko
print(f"\n💰 Fetching live ETH price from CoinGecko...")
import requests

try:
    response = requests.get(
        "https://api.coingecko.com/api/v3/simple/price",
        params={"ids": "ethereum", "vs_currencies": "usd"}
    )
    response.raise_for_status()
    eth_price_usd = response.json()["ethereum"]["usd"]
    print(f"  ✓ ETH Price: ${eth_price_usd:,.2f}")
except Exception as e:
    print(f"  ✗ Error fetching ETH price: {e}")
    print(f"  ⚠️ Falling back to approximate price: $2400")
    eth_price_usd = 2400

In [ ]:
# Simplified approach: Just show pool info and basic metrics
def get_v3_pool_info(pool_address, w3):
    """
    Get basic pool information and calculate some useful metrics.
    """
    # Pool ABI
    pool_abi = [
        {
            "inputs": [],
            "name": "liquidity",
            "outputs": [{"internalType": "uint128", "name": "", "type": "uint128"}],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "slot0",
            "outputs": [
                {"internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
                {"internalType": "int24", "name": "tick", "type": "int24"},
                {"internalType": "uint16", "name": "observationIndex", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinality", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinalityNext", "type": "uint16"},
                {"internalType": "uint8", "name": "feeProtocol", "type": "uint8"},
                {"internalType": "bool", "name": "unlocked", "type": "bool"}
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "token0",
            "outputs": [{"internalType": "address", "name": "", "type": "address"}],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "token1",
            "outputs": [{"internalType": "address", "name": "", "type": "address"}],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "tickSpacing",
            "outputs": [{"internalType": "int24", "name": "", "type": "int24"}],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "fee",
            "outputs": [{"internalType": "uint24", "name": "", "type": "uint24"}],
            "stateMutability": "view",
            "type": "function"
        }
    ]

    erc20_abi = [
        {
            "inputs": [{"internalType": "address", "name": "account", "type": "address"}],
            "name": "balanceOf",
            "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "symbol",
            "outputs": [{"internalType": "string", "name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "decimals",
            "outputs": [{"internalType": "uint8", "name": "", "type": "uint8"}],
            "stateMutability": "view",
            "type": "function"
        }
    ]

    pool = w3.eth.contract(address=Web3.to_checksum_address(pool_address), abi=pool_abi)
    
    # Get pool state
    slot0 = pool.functions.slot0().call()
    current_tick = slot0[1]
    sqrt_price_x96 = slot0[0]
    liquidity = pool.functions.liquidity().call()
    tick_spacing = pool.functions.tickSpacing().call()
    fee = pool.functions.fee().call()
    
    # Get token info
    token0_address = pool.functions.token0().call()
    token1_address = pool.functions.token1().call()
    
    token0 = w3.eth.contract(address=Web3.to_checksum_address(token0_address), abi=erc20_abi)
    token1 = w3.eth.contract(address=Web3.to_checksum_address(token1_address), abi=erc20_abi)
    
    token0_symbol = token0.functions.symbol().call()
    token1_symbol = token1.functions.symbol().call()
    token0_decimals = token0.functions.decimals().call()
    token1_decimals = token1.functions.decimals().call()
    
    # Get token balances in pool
    token0_balance = token0.functions.balanceOf(pool_address).call()
    token1_balance = token1.functions.balanceOf(pool_address).call()
    
    # Calculate current price
    sqrt_price = sqrt_price_x96 / (2 ** 96)
    price = sqrt_price ** 2
    price_adjusted = price * (10 ** token0_decimals) / (10 ** token1_decimals)
    
    # Convert balances to human readable
    token0_balance_human = token0_balance / (10 ** token0_decimals)
    token1_balance_human = token1_balance / (10 ** token1_decimals)
    
    return {
        'pool_address': pool_address,
        'token0_symbol': token0_symbol,
        'token1_symbol': token1_symbol,
        'token0_address': token0_address,
        'token1_address': token1_address,
        'token0_decimals': token0_decimals,
        'token1_decimals': token1_decimals,
        'token0_balance': token0_balance_human,
        'token1_balance': token1_balance_human,
        'price': price_adjusted,
        'current_tick': current_tick,
        'liquidity': liquidity,
        'tick_spacing': tick_spacing,
        'fee': fee
    }

# Get pool info for all pools
pools_info = []

for pool_info in all_pools:
    print(f"\nGetting info for {pool_info['pair']} @ {pool_info['fee']/10000}%...")
    try:
        info = get_v3_pool_info(pool_info['address'], w3)
        info['pair'] = pool_info['pair']
        pools_info.append(info)
        print(f"  ✓ Price: {info['price']:.6e} {info['token1_symbol']}/{info['token0_symbol']}")
        print(f"  ✓ Reserves: {info['token0_balance']:.4f} {info['token0_symbol']}, {info['token1_balance']:,.2f} {info['token1_symbol']}")
        print(f"  ✓ Liquidity: {info['liquidity']:,}")
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*60}")
print(f"Successfully retrieved {len(pools_info)} pools")
print(f"{'='*60}")

In [ ]:
# Summary Report: CEX vs DEX Liquidity
print("="*80)
print("HBOT LIQUIDITY REPORT")
print("="*80)

# CEX Summary
print(f"\n📊 CENTRALIZED EXCHANGE (Bitrue)")
print("─"*80)
print(f"Trading Pair: HBOT-USDT")
print(f"Mid Price: ${(best_bid + best_ask) / 2:.6f}")
print(f"Spread: {spread_pct:.2f}%")
print(f"Bid Liquidity: {bids_df['amount'].sum():,.2f} HBOT (${bids_df['value'].sum():,.2f})")
print(f"Ask Liquidity: {asks_df['amount'].sum():,.2f} HBOT (${asks_df['value'].sum():,.2f})")
print(f"Total CEX Liquidity: ${(bids_df['value'].sum() + asks_df['value'].sum()):,.2f}")

# DEX Summary
print(f"\n🦄 DECENTRALIZED EXCHANGE (Uniswap V3)")
print("─"*80)

# Store calculated USD prices for each pool
pool_usd_prices = {}

for pool in pools_info:
    print(f"\n{pool['pair']} @ {pool['fee']/10000}%")
    print(f"  Address: {pool['pool_address']}")
    print(f"  Price: {pool['price']:.6e} {pool['token1_symbol']}/{pool['token0_symbol']}")
    print(f"  Reserves:")
    print(f"    - {pool['token0_symbol']}: {pool['token0_balance']:.4f}")
    print(f"    - {pool['token1_symbol']}: {pool['token1_balance']:,.2f}")
    
    # Calculate TVL and USD price based on pool composition
    hbot_price_usd = None
    tvl = None
    
    if pool['token0_symbol'] == 'WETH' and pool['token1_symbol'] == 'HBOT':
        # HBOT/WETH pool: price is HBOT per WETH
        # Convert to USD: (ETH price in USD) / (HBOT per WETH)
        hbot_price_usd = eth_price_usd / pool['price']
        tvl = pool['token0_balance'] * eth_price_usd * 2  # Assume roughly 50/50 split
        print(f"  Estimated TVL: ${tvl:,.2f}")
        print(f"  HBOT Price (via WETH @ ${eth_price_usd:,.2f}): ${hbot_price_usd:.6f}")
        pool_usd_prices[pool['pair']] = hbot_price_usd
        
    elif pool['token0_symbol'] == 'USDC' and pool['token1_symbol'] == 'HBOT':
        # HBOT/USDC pool: price is HBOT per USDC
        # Convert to USD: 1 / (HBOT per USDC) since USDC ≈ $1
        hbot_price_usd = 1.0 / pool['price']
        tvl = pool['token0_balance'] * 2  # USDC is already in USD
        print(f"  Estimated TVL: ${tvl:,.2f}")
        print(f"  HBOT Price (via USDC): ${hbot_price_usd:.6f}")
        pool_usd_prices[pool['pair']] = hbot_price_usd
        
    elif pool['token1_symbol'] == 'USDC' and pool['token0_symbol'] == 'HBOT':
        # USDC/HBOT pool: price is USDC per HBOT
        # This is already in USD per HBOT
        hbot_price_usd = pool['price']
        tvl = pool['token1_balance'] * 2  # USDC is already in USD
        print(f"  Estimated TVL: ${tvl:,.2f}")
        print(f"  HBOT Price (via USDC): ${hbot_price_usd:.6f}")
        pool_usd_prices[pool['pair']] = hbot_price_usd

print(f"\n{'='*80}")

# Comparison
print(f"\n📈 COMPARISON")
print("─"*80)
cex_price = (best_bid + best_ask) / 2
print(f"CEX Price (USDT-quoted): ${cex_price:.6f}")

if 'HBOT/WETH' in pool_usd_prices:
    dex_weth_price = pool_usd_prices['HBOT/WETH']
    price_diff_pct = ((dex_weth_price - cex_price) / cex_price) * 100
    print(f"DEX Price (WETH pool, ETH @ ${eth_price_usd:,.2f}): ${dex_weth_price:.6f} ({price_diff_pct:+.2f}% vs CEX)")

if 'HBOT/USDC' in pool_usd_prices:
    dex_usdc_price = pool_usd_prices['HBOT/USDC']
    price_diff_pct = ((dex_usdc_price - cex_price) / cex_price) * 100
    print(f"DEX Price (USDC pool): ${dex_usdc_price:.6f} ({price_diff_pct:+.2f}% vs CEX)")

print(f"\n{'='*80}\n")

In [ ]:
# Send report via Telegram using NotificationManager

# First, send the order book image
telegram_notifier = notification_manager.get_notifier('telegram')

if telegram_notifier and os.path.exists(orderbook_png_path):
    print("📤 Sending order book chart to Telegram...")
    
    # Create caption for order book image
    orderbook_caption = f"""📊 <b>HBOT-USDT Order Book (Bitrue CEX)</b>

💰 Mid Price: <code>${(best_bid + best_ask) / 2:.6f}</code>
📏 Spread: <code>{spread_pct:.2f}%</code>

<b>Liquidity Summary:</b>
🟢 Bids: <code>{bids_df['amount'].sum():,.0f} HBOT</code> (${bids_df['value'].sum():,.2f})
🔴 Asks: <code>{asks_df['amount'].sum():,.0f} HBOT</code> (${asks_df['value'].sum():,.2f})
💵 Total: <code>${(bids_df['value'].sum() + asks_df['value'].sum()):,.2f}</code>

<i>Order book depth ±80% from mid price</i>"""
    
    try:
        orderbook_success = await telegram_notifier.send_photo(
            orderbook_png_path,
            caption=orderbook_caption
        )
        print(f"  📊 Order Book Chart: {'✅ Sent' if orderbook_success else '❌ Failed'}")
    except Exception as e:
        print(f"  ❌ Error sending order book: {e}")

# Build DEX liquidity message with corrected USD prices
telegram_message = "<b>🦄 UNISWAP V3 DEX LIQUIDITY</b>\n"
telegram_message += "━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n\n"

for i, pool in enumerate(pools_info):
    if i > 0:
        telegram_message += "\n"
    
    telegram_message += f"<b>{pool['pair']}</b> @ {pool['fee']/10000}%\n"
    telegram_message += f"├ Address: <code>{pool['pool_address'][:10]}...{pool['pool_address'][-8:]}</code>\n"
    telegram_message += f"├ Reserves:\n"
    telegram_message += f"│  ├ {pool['token0_symbol']}: <code>{pool['token0_balance']:.4f}</code>\n"
    telegram_message += f"│  └ {pool['token1_symbol']}: <code>{pool['token1_balance']:,.0f}</code>\n"
    
    # Calculate TVL and HBOT USD price properly
    if pool['token0_symbol'] == 'WETH' and pool['token1_symbol'] == 'HBOT':
        # HBOT/WETH pool
        tvl = pool['token0_balance'] * eth_price_usd * 2
        hbot_price_usd = eth_price_usd / pool['price']
        telegram_message += f"├ TVL: <code>${tvl:,.2f}</code>\n"
        telegram_message += f"└ HBOT Price: <code>${hbot_price_usd:.6f}</code> (via WETH @ ${eth_price_usd:,.2f})\n"
        
    elif pool['token0_symbol'] == 'USDC' and pool['token1_symbol'] == 'HBOT':
        # HBOT/USDC pool: price is HBOT per USDC
        tvl = pool['token0_balance'] * 2
        hbot_price_usd = 1.0 / pool['price']
        telegram_message += f"├ TVL: <code>${tvl:,.2f}</code>\n"
        telegram_message += f"└ HBOT Price: <code>${hbot_price_usd:.6f}</code> (via USDC)\n"
        
    elif pool['token1_symbol'] == 'USDC' and pool['token0_symbol'] == 'HBOT':
        # USDC/HBOT pool: price is already USD per HBOT
        tvl = pool['token1_balance'] * 2
        hbot_price_usd = pool['price']
        telegram_message += f"├ TVL: <code>${tvl:,.2f}</code>\n"
        telegram_message += f"└ HBOT Price: <code>${hbot_price_usd:.6f}</code> (via USDC)\n"

# Comparison with proper price sources
telegram_message += "\n<b>📈 CEX vs DEX COMPARISON</b>\n"
cex_price = (best_bid + best_ask) / 2
telegram_message += f"├ CEX (USDT): <code>${cex_price:.6f}</code>\n"

if 'HBOT/WETH' in pool_usd_prices:
    dex_weth_price = pool_usd_prices['HBOT/WETH']
    price_diff_pct = ((dex_weth_price - cex_price) / cex_price) * 100
    telegram_message += f"├ DEX WETH Pool: <code>${dex_weth_price:.6f}</code> ({price_diff_pct:+.2f}%)\n"

if 'HBOT/USDC' in pool_usd_prices:
    dex_usdc_price = pool_usd_prices['HBOT/USDC']
    price_diff_pct = ((dex_usdc_price - cex_price) / cex_price) * 100
    telegram_message += f"└ DEX USDC Pool: <code>${dex_usdc_price:.6f}</code> ({price_diff_pct:+.2f}%)\n"

# Add timestamp
telegram_message += f"\n<i>📅 {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}</i>"
telegram_message += f"\n<i>💱 ETH Price: ${eth_price_usd:,.2f}</i>"

# Print message to verify
print("\nTelegram Message Preview:")
print("="*60)
print(telegram_message.replace("<b>", "").replace("</b>", "").replace("<code>", "").replace("</code>", "").replace("<i>", "").replace("</i>", ""))
print("="*60)

# Send DEX liquidity message via NotificationManager
print("\n📤 Sending DEX liquidity report...")

notification = NotificationMessage(
    title="🦄 HBOT DEX Liquidity",
    message=telegram_message,
    level="info",
)

try:
    results = await notification_manager.send_notification(notification)
    
    print("\n📊 Notification Delivery Results:")
    print("-" * 40)
    for service, success in results.items():
        status_emoji = "✅" if success else "❌"
        status_text = "Delivered" if success else "Failed"
        print(f"  {status_emoji} {service.capitalize()}: {status_text}")
    
    # Summary
    successful_deliveries = sum(results.values())
    total_services = len(results)
    print(f"\n📈 Delivery Summary: {successful_deliveries}/{total_services} services successful")
    
    if successful_deliveries == 0:
        print("⚠️ No notifications were delivered successfully")
        print("💡 Check your notification configuration in .env file")
    elif 'telegram' in results and results['telegram']:
        print("🎉 HBOT liquidity report successfully sent to Telegram!")
    
except Exception as e:
    print(f"❌ Error sending notification: {e}")
    print("💡 Check your Telegram bot configuration and network connection")